In [13]:
import numpy as np
import matplotlib.pyplot as plt
from skyfield.api import load, Topos
from skyfield.data import mpc
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
from skyfield.api import Angle
from skyfield.positionlib import ICRF, build_position
from skyfield.framelib import ecliptic_frame
# Carregar efemérides e tempo
ts = load.timescale()
eph = load("de421.bsp")
t = ts.utc(2025, 7, 14, 21)  # Data e hora desejada

# Observador
observer = Topos(latitude_degrees=-7.23, longitude_degrees=-35.88)  # Ex: Campina Grande
obs = eph["earth"] + observer

# Sol e Lua
sun = eph["sun"]
moon = eph["moon"]
sun_apparent = obs.at(t).observe(sun).apparent()
moon_apparent = obs.at(t).observe(moon).apparent()

alt_sun, az_sun, _ = sun_apparent.altaz()
alt_moon, az_moon, _ = moon_apparent.altaz()

# Eclíptica
# Gerar coordenadas da eclíptica

from skyfield.api import load
from skyfield.framelib import ecliptic_frame
from skyfield.positionlib import position_of_radec

ts = load.timescale()
eph = load("de421.bsp")
t = ts.utc(2025, 7, 14, 21)
observer = eph["earth"] + Topos(latitude_degrees=-7.23, longitude_degrees=-35.88)

ecl_coords = []

for lam_deg in np.linspace(0, 360, 360):
    lam_rad = np.radians(lam_deg)
    beta_rad = 0.0

    # Unidade vetorial na eclíptica
    x = np.cos(beta_rad) * np.cos(lam_rad)
    y = np.cos(beta_rad) * np.sin(lam_rad)
    z = np.sin(beta_rad)

    # Criar vetor no sistema eclíptico
    from skyfield.positionlib import ICRF

    vec = ICRF((x, y, z), t=t).frame_latlon(ecliptic_frame)

    # Observar do ponto do observador
    app = observer.at(t).observe(vec)#.apparent()
    alt, az, _ = app.altaz()

    if alt.degrees > 0:
        ecl_coords.append((az.radians, 90 - alt.degrees))
        
# Estrelas brilhantes (Tycho-2 via Vizier)
viz = Vizier(columns=["RAJ2000", "DEJ2000", "Vmag"], column_filters={"Vmag": "<2.5"})
stars = viz.query_constraints(catalog="I/259/tyc2")[0]

# Converter para AltAz
coords = SkyCoord(ra=stars["RAJ2000"], dec=stars["DEJ2000"], unit=(u.deg, u.deg))
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz

cg = EarthLocation(lat=-7.23 * u.deg, lon=-35.88 * u.deg)
time = Time(t.utc_datetime())
frame = AltAz(obstime=time, location=cg)
altaz = coords.transform_to(frame)
bright = altaz[altaz.alt > 0]

# Plot
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111, polar=True)
ax.set_theta_zero_location("S")  # Norte em cima
ax.set_theta_direction(-1)  # Az sentido horário
ax.set_rlim(90, 0)  # Zênite = centro

# Grade horizontal
ax.set_xticks(np.radians(np.arange(0, 360, 30)))
ax.set_xticklabels(["N", "NE", "E", "SE", "S", "SO", "O", "NO", "N", "", "", ""])

# Eclíptica
az_ecl, r_ecl = zip(*ecl_coords)
ax.plot(az_ecl, r_ecl, color="orange", label="Eclíptica")

# Sol e Lua
ax.plot(
    az_sun.radians, 90 - alt_sun.degrees, "o", color="gold", label="Sol", markersize=10
)
ax.plot(
    az_moon.radians, 90 - alt_moon.degrees, "o", color="gray", label="Lua", markersize=8
)

# Estrelas brilhantes
ax.scatter(
    bright.az.radian, 90 - bright.alt.deg, s=10, color="white", edgecolor="black"
)

# Fundo
ax.set_facecolor("midnightblue")
fig.patch.set_facecolor("black")

# Constelações do zodíaco (apenas nomes e marcações simples)
zodiac = [
    "Aries",
    "Taurus",
    "Gemini",
    "Cancer",
    "Leo",
    "Virgo",
    "Libra",
    "Scorpius",
    "Sagittarius",
    "Capricornus",
    "Aquarius",
    "Pisces",
]

from astroquery.skyview import SkyView
from astropy.coordinates import get_constellation

# Marcar os centros das constelações zodiacais com símbolos
for name in zodiac:
    c = SkyCoord.from_name(name)
    c_altaz = c.transform_to(frame)
    if c_altaz.alt.deg > 0:
        ax.text(
            c_altaz.az.radian,
            90 - c_altaz.alt.deg,
            name,
            color="cyan",
            ha="center",
            fontsize=8,
        )

# Legenda
ax.legend(loc="upper right", bbox_to_anchor=(1.2, 1.1))

plt.title("Sky Chart com Eclíptica, Sol, Lua e Constelações Zodiacais", color="white")
plt.tight_layout()
plt.show()


AttributeError: 'tuple' object has no attribute '_observe_from_bcrs'

In [ ]:
ICRF.frame_latlon()